In [12]:
pip install streamlit openai==0.28 langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.26.0
    Uninstalling openai-1.26.0:
      Successfully uninstalled openai-1.26.0


In [9]:
!pip install --upgrade langchain

In [46]:
pip install langchain-openai

  Using cached openai-1.26.0-py3-none-any.whl (314 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [47]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [48]:
import yaml

file_path = "/content/chatgpt_ai_credentials.yaml"

with open(file_path, 'r') as file:
    api_creds = yaml.safe_load(file)

In [49]:
api_creds.keys()

dict_keys(['openai_key'])

In [56]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

In [84]:
# the following line is a magic command
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly
%%writefile app.py

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from operator import itemgetter

import os
import streamlit as st
import openai
from langchain import LLMChain
from langchain_openai import OpenAI

# Check for the OpenAI API key in the environment
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")

# Using an existing PromptTemplate provided by langchain
from langchain.prompts import ChatPromptTemplate

def generate_marketing_content(topic, format_type , emotion="positive", length="medium"):
    # Create input variables
    input_variables = {
        "topic": "Generative AI",
        "format_type": "linkedin post",
        "emotion": "positive",
        "length": "medium",
    }

    # Create the prompt template
    prompt = ChatPromptTemplate(input_variables=input_variables)

    # Create the LLM chain
    chain = LLMChain(llm=OpenAI(api_key=openai_api_key), prompt=prompt)

    # Generate the marketing content
    response = chain.run(f"Generate a {emotion} {format_type} about {topic} that is {length} in length.")
    return response

# Streamlit UI
def main():
    st.title('Marketing Content Generator')
    topic = st.text_input('Enter the topic:')
    format_type = st.selectbox('Select the format type:', ['blog post', 'linkedin post', 'twitter thread'])
    emotion = st.selectbox('Select the emotional tone:', ['positive', 'neutral', 'negative'])
    length = st.selectbox('Select the length:', ['short', 'medium', 'long'])

    if st.button('Generate Content'):
        content = generate_marketing_content(topic, format_type, emotion, length)
        st.text_area("Generated Content:", content, height=250)

if __name__ == '__main__':
    main()


Overwriting app.py


In [3]:
!pip install pyngrok==7.1.5

In [10]:
!streamlit run app.py --server.port=8443 &>./logs.txt &

In [12]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8443)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://6e4a-35-245-48-201.ngrok-free.app
